In [75]:
import re
import subprocess
import StringIO
import re

import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 

pdb_path = '/home/tongwade780/pdb_website/onebyone_cluster_code/build_pdb_alignment/'

pdbname_file = open ('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/recalculate_pdbs.csv').readlines ()

next_file = []

for line in pdbname_file[0:]:
    next_file.append(line.rstrip('\r\n' + '').split(','))

frames = []
problem_pdb = []
for pdb_cluster in next_file:
    pdbname = pdb_cluster[0]
    cluster = pdb_cluster[1]
#     f = open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_test.m', 'r+')
#     text = f.read()


#     text = re.sub('cluster_PDBname', pdbname, text)
#     text = re.sub('cluster_number', cluster, text)
#     f.seek(0)
#     f.close()

#     f2= open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_new.m','w+')
#     f2.write(text)
#     f2.close()
#     #run matlab code for one pdb
#     subprocess.check_output(["matlab","-nosplash","nodesktop","-wait","-r","run('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_new.m')","quit;"])


    try:
        pdb_file = '{0}{1}'.format(pdbname,'.pdb')
        pdbPATH = os.path.join(pdb_path,pdb_file)
    except:
        problem_pdb.append((pdbname,cluster,'not_have_pdb broke'))
        continue

    #sasa table
    sasa_result = getSASA_final(pdbPATH)
#     try:
    #dssp table
    final_dssp_table = getDSSP_final(pdbPATH)
#     except:
#         problem_pdb.append((pdbname,cluster,'getdssp broke'))
#         continue
    try:
        #pka table
        pka_final_result = getcharge_pKa(pdbPATH)
    except:
        problem_pdb.append((pdbname,cluster,'getcharge_pKa broke'))
        continue
        #load matlab data with dataframe 
    try:
        f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix/{0}_individual_feature.mat'.format(pdbname),'r') 
    except:
        problem_pdb.append((pdbname,cluster,'no individual_feature'))
        continue        
        
    datas = f['individual_feature'].value.T
    matlab_data = pd.DataFrame(datas,index=datas[:,0],columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                              'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                              'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                              'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                              'Polarity','charge','stand_Dcom','side_chain_length',
                                              'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                              'phobic_percent','RMSF','RMSF_0f_bfactor','shifted_RMSF',
                                                              'h_bond_group','intrinsicDisorder'])

    #change resID value type to int
    matlab_data['resID'] = matlab_data['resID'].astype('int')

    sasa_result.columns = ['resID','sasa']

    sasa_result['resID'] = sasa_result['resID'].astype('int')

    pka_final_result.columns = ['resID','pka_charge']
    pka_final_result['resID'] = pka_final_result['resID'].astype('int')
    final_dssp_table.columns = ['resID','dssp_result','loop_percent']
    final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
    final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
    # try:
    #     final_dssp_table['resID'] = sasa_result['resID'].values
    # except:
    #     problem_pdb.append((pdbname,'get_dssp_merge broke'))
    #     continue    
    final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')

    #merge matlab feature and python feature
    merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

    merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

    merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

    merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
    #add pdb id
    merged_data['PDB_ID'] = pdbname
    #change feature order
    change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                    'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                    'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                    'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                    'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                    'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                    'loop_percent','h_bond_group','intrinsicDisorder','RMSF','RMSF_0f_bfactor',
                    'shifted_RMSF']

    merged_data = merged_data[change_order]
    frames.append(merged_data)

if len(problem_pdb) == 0:
    print 'pdb_no_problem'
else:    
    problem_pdb_data=pd.DataFrame(problem_pdb)
    problem_pdb_data.columns = ['pdbname','cluster','problem']

    problem_pdb_data.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_problem_pdb_new/problem_pdb_data_builded.csv',index= False)
if len(frames) == 0:
    all_cluster_pdb = filter(None, next_file[cluster])

    all_cluster_pdb = pd.DataFrame(all_cluster_pdb)
    all_cluster_pdb.to_csv('/home/tongwade780/pdb_website/python_code/no_pdb_clsuter_new/builded_no_pdb.csv',index = False)

else:
    result = pd.concat(frames)
    result.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_feature_csv_new/builded_all_feature.csv',index = False)    

propka3.1                                                                                    2017-10-17
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

CalledProcessError: Command '['/opt/dssp/dssp', '/home/tongwade780/pdb_website/onebyone_cluster_code/build_pdb_alignment/5enk.pdb']' returned non-zero exit status 1

In [74]:
problem_pdb_data

,pdbname,cluster,problem
0,5enk,6,getdssp broke
1,5kr8,6,getdssp broke
2,1gpn,17,getdssp broke
3,1vot,17,getdssp broke
4,2w6c,17,getdssp broke
5,1h23,17,getdssp broke
6,1acj,17,getdssp broke
7,2ace,17,getdssp broke
8,2ack,17,getdssp broke
9,1ax9,17,getdssp broke


In [72]:
pdb_cluster[1]

'2769'

In [38]:
import re
f = open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_test.m', 'r+')
text = f.read()

In [45]:
import re
f = open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_test.m', 'r+')
text = f.read()


text = re.sub('cluster_PDBname', next_file[0][0], text)
text = re.sub('cluster_number', next_file[0][1], text)
f.seek(0)
f.close()

f2= open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_new.m','w+')
f2.write(text)
f2.close()

In [48]:
import subprocess
import StringIO
subprocess.check_output(["matlab","-nosplash","nodesktop","-wait","-r","run('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/load_one_pdb_feature_new.m')","quit;"])


propka3.1                                                                                    2017-09-22
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

In [52]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 
#pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdb_path = '/home/tongwade780/pdb_website/onebyone_cluster_code/build_pdb_alignment/'
frames = []
problem_pdb = []


pdbname= next_file[0][0]
clsuter  = next_file[0][1]
pdb_file = '{0}{1}'.format(pdbname,'.pdb')
pdbPATH = os.path.join(pdb_path,pdb_file)

#sasa table
sasa_result = getSASA_final(pdbPATH)
#dssp table
final_dssp_table = getDSSP_final(pdbPATH)
try:
    #pka table
    pka_final_result = getcharge_pKa(pdbPATH)
except:
    problem_pdb.append((pdbname,clsuter,'getcharge_pKa broke'))
#     continue
    #load matlab data with dataframe 

f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix/{0}_individual_feature.mat'.format(pdbname),'r') 

datas = f['individual_feature'].value.T
matlab_data = pd.DataFrame(datas,index=datas[:,0],columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                          'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                          'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                          'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                          'Polarity','charge','stand_Dcom','side_chain_length',
                                          'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                          'phobic_percent','RMSF','RMSF_0f_bfactor','shifted_RMSF',
                                                          'h_bond_group','intrinsicDisorder'])

#change resID value type to int
matlab_data['resID'] = matlab_data['resID'].astype('int')

sasa_result.columns = ['resID','sasa']

sasa_result['resID'] = sasa_result['resID'].astype('int')

pka_final_result.columns = ['resID','pka_charge']
pka_final_result['resID'] = pka_final_result['resID'].astype('int')
final_dssp_table.columns = ['resID','dssp_result','loop_percent']
final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
# try:
#     final_dssp_table['resID'] = sasa_result['resID'].values
# except:
#     problem_pdb.append((pdbname,'get_dssp_merge broke'))
#     continue    
final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')

#merge matlab feature and python feature
merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
#add pdb id
merged_data['PDB_ID'] = pdbname
#change feature order
change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                'loop_percent','h_bond_group','intrinsicDisorder','RMSF','RMSF_0f_bfactor',
                'shifted_RMSF']

merged_data = merged_data[change_order]
frames.append(merged_data)

if len(problem_pdb) == 0:
    print 'pdb_no_problem'
else:    
    problem_pdb_data=pd.DataFrame(problem_pdb)
    problem_pdb_data.columns = ['pdbname','probelm']

    problem_pdb_data.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_problem_pdb_new/problem_pdb_data_builded.csv',index= False)
if len(frames) == 0:
    all_cluster_pdb = filter(None, next_file[cluster])

    all_cluster_pdb = pd.DataFrame(all_cluster_pdb)
    all_cluster_pdb.to_csv('/home/tongwade780/pdb_website/python_code/no_pdb_clsuter_new/builded_no_pdb.csv',index = False)
        
else:
    result = pd.concat(frames)
    result.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_feature_csv_new/builded_all_feature.csv',index = False)

propka3.1                                                                                    2017-09-22
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

,PDB_ID,resID,atomic_1,atomic_3,atomic_5,ssbond_matrix,Polarity,eig_vv_1,eig_vv_2,vector_1,...,shannon_entropy,shannon_entropy20,eig5_eig1,phobic_percent,loop_percent,h_bond_group,intrinsicDisorder,RMSF,RMSF_0f_bfactor,shifted_RMSF
0,4y35,1,19.0,16.000000,15.333333,0.0,9.2,0.160320,0.246952,0.018968,...,0.927013,0.910987,0.711923,56.969697,44.545455,4.0,0.567626,0.145033,0.691520,0.631727
1,4y35,2,20.0,15.333333,12.500000,0.0,8.6,0.133419,0.253021,0.015785,...,0.927013,0.910987,0.711923,56.969697,44.545455,0.0,0.575627,0.123037,0.605940,0.609731
2,4y35,3,22.0,12.666667,12.000000,0.0,9.0,0.167331,0.207627,0.019797,...,0.927013,0.910987,0.711923,56.969697,44.545455,2.0,0.630163,0.120206,0.571787,0.606900
3,4y35,4,12.0,11.666667,11.400000,0.0,9.2,0.137599,0.198383,0.016280,...,0.927013,0.910987,0.711923,56.969697,44.545455,2.0,0.641117,0.105791,0.558093,0.592484
4,4y35,5,16.0,10.000000,12.400000,0.0,8.1,0.133062,0.171847,0.015743,...,0.927013,0.910987,0.711923,56.969697,44.545455,2.0,0.619954,0.096871,0.550409,0.583564
5,4y35,6,15.0,13.666667,10.400000,0.0,8.6,0.080293,0.155688,0.009500,...,0.927013,0.910987,0.711923,56.969697,44.545455,0.0,0.618181,0.120146,0.526991,0.606840
6,4y35,7,31.0,11.666667,11.000000,0.0,8.6,0.070233,0.127656,0.008309,...,0.927013,0.910987,0.711923,56.969697,44.545455,5.0,0.601823,0.100864,0.540159,0.587558
7,4y35,8,20.0,13.666667,12.400000,0.0,8.6,0.075116,0.094338,0.008887,...,0.927013,0.910987,0.711923,56.969697,44.545455,2.0,0.604820,0.111631,0.575904,0.598325
8,4y35,9,19.0,14.000000,11.800000,0.0,8.0,0.028747,0.063403,0.003401,...,0.927013,0.910987,0.711923,56.969697,44.545455,0.0,0.592304,0.119567,0.584378,0.606261
9,4y35,10,24.0,14.000000,8.400000,0.0,5.2,0.077408,0.028632,0.009158,...,0.927013,0.910987,0.711923,56.969697,44.545455,1.0,0.571123,0.113237,0.598800,0.599931
